In [1]:
# Importing Relevant Packages
from ortools.sat.python import cp_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import math
import random
import re
import csv
import unicodedata
from collections import defaultdict
from dataclasses import dataclass
from typing import List, Optional
from statistics import stdev
from fpdf import FPDF
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.optimize import minimize

load C:\Users\andri\anaconda3\Lib\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\andri\anaconda3\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\andri\anaconda3\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\andri\anaconda3\Lib\site-packages\ortools\.libs\re2.dll...
load C:\Users\andri\anaconda3\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\andri\anaconda3\Lib\site-packages\ortools\.libs\highs.dll...
load C:\Users\andri\anaconda3\Lib\site-packages\ortools\.libs\ortools.dll...


In [3]:
@dataclass
class Patient: # Forming the structure for the patients dataset
    id: int # Unique patient identifier
    name: str # Anonymised patient label
    age: int # Age of patient in yyears
    gender: str # Gender of patient
    admission_day: int # Day the patient was admitted in the hospital
    release_day: int # Day the patient was released from the hospital
    spec_id: int # ID of the patient's specialism
    spec_days: int # Number of days a patient requires this specialism
    prefered_capacity: int # How many number of beds it is preffered the room to have
    needs: List[int] # ID of features the patient needs
    prefers: List[int] # ID of features the patient prefers

@dataclass
class Room: # Forming the structure for the room dataset
    id: int # Unique room identifier
    room_number: int # Room number in hospital
    department: int # ID of the department the room belongs
    capacity: int # How many beds the room has
    gender: str # Gender the room is designated for
    has: List[int] # Equipment the room has
    spec_ids: List[int] # IDs of the specialisms supported by this room
    penalties: List[int] # Penalties of specialisms

In [5]:
# Function to build hospital data from CSV files
def build_hospital_data():
    # Read room and patient data from CSV files into pandas DataFrames
    rooms_df = pd.read_csv("cleaned_rooms1.csv")
    patients_df = pd.read_csv("cleaned_patients1.csv")

    # Create a list of Room objects from the rooms DataFrame
    rooms = [
        Room(
            id=i, # Assign a unique ID to each room
            room_number=int(row["RoomNumber"]), # Get room number
            department=int(row["DepartmentID"]), # Get department ID
            capacity=int(row["Capacity"]), # Get room capacity
            gender=row["Gender"], # Get gender designation
            has=[ # List of equipment the room has (assuming order: Telemetry, Oxygen, Nitrogen, Television)
                int(row["Telemetry"]),
                int(row["Oxygen"]),
                int(row["Nitrogen"]),
                int(row["Television"])
            ],
            spec_ids=[ # List of specialism IDs supported by the room
                int(row["ReqSpecialism1"]),
                int(row["ReqSpecialism2"]),
                int(row["ReqSpecialism3"])
            ],
            penalties=[ # List of penalties associated with specialisms in this room
                int(row["PenaltySpecialism1"]),
                int(row["PenaltySpecialism2"]),
                int(row["PenaltySpecialism3"])
            ]
        )
        for i, row in rooms_df.iterrows() # Iterate through each row of the rooms DataFrame
    ]

    # Create a list of Patient objects from the patients DataFrame
    patients = [
        Patient(
            id=i, # Assign a unique ID to each patient
            name=row["Name"], # Get patient name
            age=int(row["Age"]), # Get patient age
            gender=row["Gender"], # Get patient gender
            admission_day=int(row["AdmissionDay"]), # Get admission day
            release_day=int(row["ReleaseDay"]), # Get release day
            spec_id=int(row["SpecialismID"]), # Get specialism ID
            spec_days=int(row["SpecialismDays"]), # Get number of days specialism is needed
            prefered_capacity=int(row["PreferredRoomCapacity"]), # Get preferred room capacity
            needs=[ # List of equipment the patient needs (assuming order: Telemetry, Oxygen, Nitrogen, Television)
                int(row["NeedsTelemetry"]),
                int(row["NeedsOxygen"]),
                int(row["NeedsNitrogen"]),
                int(row["NeedsTV"])
            ],
            prefers=[ # List of equipment the patient prefers (assuming order: Telemetry, Oxygen, Nitrogen, Television)
                int(row["PrefersTelemetry"]),
                int(row["PrefersOxygen"]),
                int(row["PrefersNitrogen"]),
                int(row["PrefersTV"])
            ]
        )
        for i, row in patients_df.iterrows() # Iterate through each row of the patients DataFrame
    ]

    # Return the lists of Room and Patient objects
    return rooms, patients

In [7]:
# Custom solution callback for printing the final hospital allocation
class HospitalAllocationPrinter(cp_model.CpSolverSolutionCallback):
    # Constructor to initialize the callback with necessary data
    def __init__(self, seats, patients, rooms):
        super().__init__() # Call the constructor of the parent class
        self.__seats = seats # Store the dictionary of boolean variables representing assignments
        self.__patients = patients # Store the list of patient objects
        self.__rooms = rooms # Store the list of room objects
        self.__solution_found = False # Flag to indicate if a solution has been found

    # Callback method called by the solver when an improving solution is found
    def on_solution_callback(self):
        self.__solution_found = True # Set the flag to True

    # Method to print the final solution after the solver finishes
    def print_final_solution(self, solver):
        print("\nFinal solution:") # Print a header for the final solution
        # Iterate through each room to display its assignments
        for room in self.__rooms:
            # Print room details
            print(f"\nRoom {room.id} (RoomNum: {room.room_number}, Dept: {room.department}, Capacity: {room.capacity}, Gender: {room.gender}):")
            assigned = False # Flag to check if any patient is assigned to this room
            # Iterate through each patient to see if they are assigned to the current room
            for patient in self.__patients:
                # Check if the boolean variable for this patient-room assignment is True in the final solution
                if solver.BooleanValue(self.__seats[(room.id, patient.id)]):
                    # Print patient details if assigned to this room
                    print(f"  - Patient {patient.name} (Age: {patient.age}, Gender: {patient.gender})")
                    assigned = True # Set flag to True as a patient is assigned
            # If no patient was assigned to this room, print "(empty)"
            if not assigned:
                print("  (empty)")

    # Method to check if a solution has been found
    def solution_found(self):
        return self.__solution_found

In [9]:
# Custom solution callback to log improving solutions found by the CP-SAT solver
class ImprovingSolutionLogger(cp_model.CpSolverSolutionCallback):
    # Constructor to initialize the logger
    def __init__(self):
        super().__init__() # Call the constructor of the parent class
        self.times = [] # List to store the elapsed time when an improving solution is found
        self.objectives = [] # List to store the objective value of improving solutions
        self.start_time = time.time() # Record the start time when the logger is initialized

    # Callback method called by the solver when an improving solution is found
    def on_solution_callback(self):
        elapsed = time.time() - self.start_time # Calculate the elapsed time since the start
        current_obj = self.ObjectiveValue() # Get the objective value of the current solution
        self.times.append(elapsed) # Append the elapsed time to the list
        self.objectives.append(current_obj) # Append the objective value to the list
        # Print the objective value and elapsed time for the improving solution
        print(f"Objective {current_obj} at {elapsed:.2f}s")

In [11]:
# Main function to solve the hospital allocation problem
def solve_hospital_allocation():
    # Build hospital data (rooms and patients)
    rooms, patients = build_hospital_data()

    # Determine the maximum admission day to define the planning horizon
    max_day = max(p.admission_day for p in patients)
    previous_assignment = {} # Dictionary to store the previous room assignment for each patient
    total_duration = 0 # Variable to accumulate the total solving time across all days

    all_day_logs = {}  # Dictionary to store improving solutions and status per day

    # Iterate through each day in the planning horizon
    for day in range(max_day + 1):
        print(f"\n Solving for Day {day}") # Print the current day being solved
        # Identify patients present on the current day (admission <= current day < release)
        present_patients = [p for p in patients if p.admission_day <= day < p.release_day]
        print(f"Patients present: {len(present_patients)}") # Print the number of patients present

        if not present_patients: # If no patients are present, skip to the next day
            continue

        # CP-SAT Model Definition
        model = cp_model.CpModel() # Create a new CP-SAT model
        seats = {
            # Create a boolean variable for each possible patient-room assignment on the current day
            (r.id, p.id): model.NewBoolVar(f"patient_{p.id}_in_room_{r.id}_day{day}")
            for r in rooms for p in present_patients
        }

        # Soft Constraint Violations - Lists to store boolean variables for each type of violation
        violations_preferred_capacity = [] # Violations of preferred room capacity
        violations_gender = [] # Violations of gender preference
        violations_equipment = [] # Violations of preferred equipment (re-used, consider renaming for clarity)
        equipment_violations = [] # Violations of needed equipment
        move_penalties = [] # Penalties for moving a patient to a different room
        age_violations = [] # Violations of department age limits
        violations_specialism = [] # Violations related to specialism penalties

        # Populate the violation lists based on potential assignments and penalties
        for p in present_patients:
            prev_room = previous_assignment.get(p.id, None) # Get the patient's previous room assignment
            for r in rooms:
                seat = seats[(r.id, p.id)] # The boolean variable for this patient-room assignment

                # Check for each type of soft constraint violation if this assignment is made
                if r.capacity > p.prefered_capacity:
                    violations_preferred_capacity.append(seat)
                if r.gender in ['F', 'M'] and r.gender != p.gender:
                    violations_gender.append(seat)
                if any(p.prefers[i] == 1 and r.has[i] == 0 for i in range(4)): # Assuming 4 equipment types
                    violations_equipment.append(seat)
                if any(p.needs[i] == 1 and r.has[i] == 0 for i in range(4)): # Assuming 4 equipment types
                    equipment_violations.append(seat)
                if prev_room is not None and r.id != prev_room:
                    move_penalties.append(seat)
                # Age violations based on department (assuming specific department IDs and age limits)
                if (r.department == 1 and p.age < 65) or (r.department == 4 and p.age > 16):
                    age_violations.append(seat)
                # Specialism violations based on room's supported specialisms and penalties
                if p.spec_id in r.spec_ids:
                    penalty = r.penalties[r.spec_ids.index(p.spec_id)] - 1 # Get the penalty for the specialism
                    if penalty > 0: # Add penalty proportional to the penalty value
                        violations_specialism.extend([seat] * penalty)
                else: # Add a larger penalty if the specialism is not supported
                    violations_specialism.extend([seat] * 3)

        # Hard Constraints
        # Each patient must be assigned to exactly one room
        for p in present_patients:
            model.Add(sum(seats[(r.id, p.id)] for r in rooms) == 1)
        # Each room's capacity must not be exceeded
        for r in rooms:
            model.Add(sum(seats[(r.id, p.id)] for p in present_patients) <= r.capacity)

        # Define the objective function to minimize the total weighted sum of soft constraint violations
        model.Minimize(
            3 * sum(violations_preferred_capacity) + # Weight for preferred capacity violations
            5 * sum(violations_gender) + # Weight for gender violations
            3 * sum(violations_equipment) + # Weight for preferred equipment violations
            7 * sum(age_violations) + # Weight for age violations
            10 * sum(move_penalties) + # Weight for move penalties
            10 * sum(equipment_violations) + # Weight for needed equipment violations
            4 * sum(violations_specialism) # Weight for specialism violations
        )

        # Solve the model for the current day
        solver = cp_model.CpSolver() # Create a CP-SAT solver instance
        solver.parameters.max_time_in_seconds = 500  # Set a reasonable time limit for solving
        logger = ImprovingSolutionLogger() # Create a logger to track improving solutions

        start_time = time.time() # Record the start time for solving the day
        # Solve the model using the logger callback
        status = solver.SolveWithSolutionCallback(model, logger)
        end_time = time.time() # Record the end time for solving the day

        duration = end_time - start_time # Calculate the time taken to solve the day
        total_duration += duration # Add the day's duration to the total duration

        # Print the best objective value found and the time taken
        if logger.objectives:
            print(f"Best objective for Day {day}: {logger.objectives[-1]} in {logger.times[-1]:.2f}s")
        else:
            print("No feasible solution found.") # Print message if no feasible solution is found

        # Print the final solution for the current day
        printer = HospitalAllocationPrinter(seats, present_patients, rooms)
        printer.print_final_solution(solver)

        # Store the current assignments to handle move penalties for the next day
        current_assignment = {
            p.id: r.id
            for r in rooms
            for p in present_patients
            if solver.BooleanValue(seats[(r.id, p.id)]) # Check if the patient is assigned to this room
        }
        previous_assignment = current_assignment # Update previous assignment for the next day

        # Save per-day log (times and objectives of improving solutions, and final status)
        all_day_logs[day] = {
            "times": logger.times,
            "objectives": logger.objectives,
            "status": solver.StatusName(status)
        }

    print(f"\nTotal solving time across all days: {total_duration:.2f} seconds.") # Print the total solving time
    return all_day_logs # Return the logs for all days

In [ ]:
if __name__ == "__main__":
    # Solve the hospital allocation problem and get the logs
    logs = solve_hospital_allocation()

    # Extract best (final) objective for each day
    days = [] # List to store day numbers
    penalties = [] # List to store the best objective value (penalty) for each day

    # Iterate through the logs for each day
    for day, data in logs.items():
        if data["objectives"]:  # Check if there were any improving solutions found for the day (i.e., a feasible solution)
            days.append(day) # Add the day number to the list
            penalties.append(data["objectives"][-1])  # Append the last (best found) objective value for the day


    # Optional plot for Day 0 (or any other specific day, currently set to day 19)
    import matplotlib.pyplot as plt # Import matplotlib for plotting
    day_to_plot = 19 # Define the specific day to plot
    # Check if logs exist for the specified day and if there are objectives (solutions) for that day
    if day_to_plot in logs and logs[day_to_plot]["objectives"]:
        day_data = logs[day_to_plot] # Get the data for the specified day
        # Plot the objective value progress over time for the selected day
        plt.plot(day_data["times"], day_data["objectives"], marker='o') # Create a line plot with markers
        plt.xlabel("Time (s)") # Set the x-axis label
        plt.ylabel("Objective Value") # Set the y-axis label
        plt.title(f"Optimization Progress Over Time (Day {day_to_plot})") # Set the title of the plot
        plt.savefig("feasible19.png", dpi=300) # Save the plot to a file
        plt.grid(True) # Add a grid to the plot
        plt.show() # Display the plot


 Solving for Day 0
Patients present: 70
Objective 1059.0 at 0.50s
Objective 556.0 at 0.71s
Objective 87.0 at 0.78s
Best objective for Day 0: 87.0 in 0.78s

Final solution:

Room 0 (RoomNum: 11, Dept: 1, Capacity: 1, Gender: F):
  - Patient Patient34 (Age: 74, Gender: F)

Room 1 (RoomNum: 12, Dept: 1, Capacity: 1, Gender: D):
  - Patient Patient39 (Age: 106, Gender: F)

Room 2 (RoomNum: 13, Dept: 1, Capacity: 1, Gender: N):
  - Patient Patient65 (Age: 103, Gender: F)

Room 3 (RoomNum: 14, Dept: 1, Capacity: 1, Gender: F):
  (empty)

Room 4 (RoomNum: 15, Dept: 1, Capacity: 2, Gender: D):
  (empty)

Room 5 (RoomNum: 16, Dept: 1, Capacity: 2, Gender: N):
  (empty)

Room 6 (RoomNum: 17, Dept: 1, Capacity: 2, Gender: D):
  (empty)

Room 7 (RoomNum: 18, Dept: 1, Capacity: 2, Gender: N):
  (empty)

Room 8 (RoomNum: 19, Dept: 1, Capacity: 2, Gender: N):
  - Patient Patient13 (Age: 71, Gender: F)

Room 9 (RoomNum: 110, Dept: 1, Capacity: 2, Gender: M):
  (empty)

Room 10 (RoomNum: 111, Dept: 1,